## NFL Picks

NFL match prediction with scores using historical data (1999-Present).

### Import Required Packages

In [ ]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("Packages imported successfully!")

### Load Historical NFL Data

In [ ]:
# Load historical data from 1999 to present
print("Loading NFL historical data from 1999 to present...")

# Get years from 1999 to current year
current_year = pd.Timestamp.now().year
years = list(range(1999, current_year + 1))

# Load play-by-play data (this might take a few minutes)
pbp_data = nfl.import_pbp_data(years)
print(f"Loaded play-by-play data: {pbp_data.shape}")

# Load schedule data
schedules = nfl.import_schedules(years)
print(f"Loaded schedules: {schedules.shape}")

# We'll derive team stats from the play-by-play and schedule data
print("Data loading complete!")

### Data Exploration and Understanding

In [ ]:
# Explore the structure of our datasets
print("=== SCHEDULE DATA SAMPLE ===")
print(schedules.head())
print("\nSchedule columns:", schedules.columns.tolist())

print("\n=== PLAY BY PLAY SAMPLE ===")
print(pbp_data.head())
print(f"\nPBP columns count: {len(pbp_data.columns)}")

# Check for missing scores
print(f"\nGames with missing scores: {schedules[['home_score', 'away_score']].isnull().sum()}")

# Look at score distributions
completed_games = schedules.dropna(subset=['home_score', 'away_score'])
print(f"\nCompleted games: {len(completed_games)}")
print(f"Average home score: {completed_games['home_score'].mean():.1f}")
print(f"Average away score: {completed_games['away_score'].mean():.1f}")
print(f"Average total score: {(completed_games['home_score'] + completed_games['away_score']).mean():.1f}")

### Create Team Performance Features

In [ ]:
# Create rolling team performance metrics
def calculate_team_performance_features(schedules_df):
    """Calculate rolling team performance metrics"""
    
    # Sort by team and date
    df = schedules_df.copy()
    df['game_date'] = pd.to_datetime(df['gameday'])
    
    # Create a list to store all team game records
    all_games = []
    
    # Process home games
    home_games = df[['season', 'week', 'game_date', 'home_team', 'home_score', 'away_score']].copy()
    home_games.columns = ['season', 'week', 'game_date', 'team', 'points_for', 'points_against']
    home_games['is_home'] = 1
    
    # Process away games
    away_games = df[['season', 'week', 'game_date', 'away_team', 'away_score', 'home_score']].copy()
    away_games.columns = ['season', 'week', 'game_date', 'team', 'points_for', 'points_against']
    away_games['is_home'] = 0
    
    # Combine all games
    all_games = pd.concat([home_games, away_games], ignore_index=True)
    all_games = all_games.sort_values(['team', 'game_date']).reset_index(drop=True)
    
    # Calculate rolling features (last 4 games)
    rolling_features = []
    
    for team in all_games['team'].unique():
        team_games = all_games[all_games['team'] == team].copy()
        
        # Calculate rolling averages
        team_games['avg_points_for_L4'] = team_games['points_for'].rolling(window=4, min_periods=1).mean().shift(1)
        team_games['avg_points_against_L4'] = team_games['points_against'].rolling(window=4, min_periods=1).mean().shift(1)
        team_games['win_rate_L4'] = ((team_games['points_for'] > team_games['points_against']).astype(int).rolling(window=4, min_periods=1).mean()).shift(1)
        
        # Calculate season stats up to current week
        team_games['season_points_for_avg'] = team_games.groupby('season')['points_for'].expanding().mean().shift(1).values
        team_games['season_points_against_avg'] = team_games.groupby('season')['points_against'].expanding().mean().shift(1).values
        
        rolling_features.append(team_games)
    
    return pd.concat(rolling_features, ignore_index=True)

print("Calculating team performance features...")
team_performance = calculate_team_performance_features(schedules)
print(f"Team performance features calculated: {team_performance.shape}")
print(team_performance.head())

### Merge Features with Game Data

In [ ]:
# Create the main dataset for modeling
def create_modeling_dataset(schedules_df, team_perf_df):
    """Merge schedule data with team performance features"""
    
    df = schedules_df.copy()
    df['game_date'] = pd.to_datetime(df['gameday'])
    
    # Merge home team features
    home_features = team_perf_df[team_perf_df['is_home'] == 1].copy()
    home_features = home_features.add_suffix('_home')
    home_features.rename(columns={
        'team_home': 'home_team',
        'season_home': 'season',
        'week_home': 'week'
    }, inplace=True)
    
    df = df.merge(
        home_features[['season', 'week', 'home_team', 'avg_points_for_L4_home', 
                      'avg_points_against_L4_home', 'win_rate_L4_home',
                      'season_points_for_avg_home', 'season_points_against_avg_home']],
        on=['season', 'week', 'home_team'],
        how='left'
    )
    
    # Merge away team features
    away_features = team_perf_df[team_perf_df['is_home'] == 0].copy()
    away_features = away_features.add_suffix('_away')
    away_features.rename(columns={
        'team_away': 'away_team',
        'season_away': 'season',
        'week_away': 'week'
    }, inplace=True)
    
    df = df.merge(
        away_features[['season', 'week', 'away_team', 'avg_points_for_L4_away',
                      'avg_points_against_L4_away', 'win_rate_L4_away',
                      'season_points_for_avg_away', 'season_points_against_avg_away']],
        on=['season', 'week', 'away_team'],
        how='left'
    )
    
    return df

print("Creating modeling dataset...")
modeling_data = create_modeling_dataset(schedules, team_performance)

# Remove games without scores (future games)
modeling_data = modeling_data.dropna(subset=['home_score', 'away_score'])

print(f"Modeling dataset shape: {modeling_data.shape}")
print(f"Date range: {modeling_data['gameday'].min()} to {modeling_data['gameday'].max()}")

### Feature Engineering and Data Preparation

In [ ]:
# Prepare features for modeling
def prepare_features(df):
    """Prepare final feature set for modeling"""
    
    # Select and create features
    feature_cols = [
        'week', 'season',
        'avg_points_for_L4_home', 'avg_points_against_L4_home', 'win_rate_L4_home',
        'avg_points_for_L4_away', 'avg_points_against_L4_away', 'win_rate_L4_away',
        'season_points_for_avg_home', 'season_points_against_avg_home',
        'season_points_for_avg_away', 'season_points_against_avg_away'
    ]
    
    # Create additional features
    df['home_strength'] = df['avg_points_for_L4_home'] - df['avg_points_against_L4_home']
    df['away_strength'] = df['avg_points_for_L4_away'] - df['avg_points_against_L4_away']
    df['strength_diff'] = df['home_strength'] - df['away_strength']
    df['win_rate_diff'] = df['win_rate_L4_home'] - df['win_rate_L4_away']
    
    feature_cols.extend(['home_strength', 'away_strength', 'strength_diff', 'win_rate_diff'])
    
    # Fill NaN values with 0 (for early season games)
    df[feature_cols] = df[feature_cols].fillna(0)
    
    return df, feature_cols

print("Preparing features...")
modeling_data, feature_columns = prepare_features(modeling_data)

# Create target variables
modeling_data['total_score'] = modeling_data['home_score'] + modeling_data['away_score']
modeling_data['score_diff'] = modeling_data['home_score'] - modeling_data['away_score']  # Positive = home win

print("Features prepared!")
print("Feature columns:", feature_columns)
print(f"Games with complete features: {len(modeling_data)}")

### Train Models for Score Prediction

In [ ]:
# Prepare training data with feature scaling
X = modeling_data[feature_columns]
y_home = modeling_data['home_score']
y_away = modeling_data['away_score']
y_total = modeling_data['total_score']

# Split data (use most recent 2 seasons as test set)
recent_cutoff = modeling_data['season'].max() - 1
train_mask = modeling_data['season'] < recent_cutoff
test_mask = modeling_data['season'] >= recent_cutoff

X_train, X_test = X[train_mask], X[test_mask]
y_home_train, y_home_test = y_home[train_mask], y_home[test_mask]
y_away_train, y_away_test = y_away[train_mask], y_away[test_mask]
y_total_train, y_total_test = y_total[train_mask], y_total[test_mask]

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Scale features for better performance (optional but recommended)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Optimized model parameters for better performance
optimal_params = {
    'n_estimators': 300,           # Increased from 100 for better learning
    'learning_rate': 0.05,         # Reduced for more careful learning
    'max_depth': 6,                # Increased from default 3 for more complexity
    'subsample': 0.8,              # Add randomness to prevent overfitting
    'max_features': 'sqrt',        # Use subset of features for each tree
    'min_samples_split': 10,       # Require more samples to split
    'min_samples_leaf': 5,         # Require more samples in leaf nodes
    'random_state': 42,
    'validation_fraction': 0.1,    # Use 10% for early stopping
    'n_iter_no_change': 20,        # Stop if no improvement for 20 rounds
    'tol': 1e-4                    # Tolerance for early stopping
}

# Train optimized models for home score, away score, and total score
models = {}
scalers = {'scaler': scaler}  # Store scaler for predictions

# Home Score Model
print("Training optimized home score model...")
models['home_score'] = GradientBoostingRegressor(**optimal_params)
models['home_score'].fit(X_train_scaled, y_home_train)

# Away Score Model
print("Training optimized away score model...")
models['away_score'] = GradientBoostingRegressor(**optimal_params)
models['away_score'].fit(X_train_scaled, y_away_train)

# Total Score Model
print("Training optimized total score model...")
models['total_score'] = GradientBoostingRegressor(**optimal_params)
models['total_score'].fit(X_train_scaled, y_total_train)

print("All optimized models trained!")
print(f"Home score model used {models['home_score'].n_estimators_} estimators")
print(f"Away score model used {models['away_score'].n_estimators_} estimators")  
print(f"Total score model used {models['total_score'].n_estimators_} estimators")

### Evaluate Model Performance

In [ ]:
# Make predictions
predictions = {}
predictions['home_score'] = models['home_score'].predict(X_test)
predictions['away_score'] = models['away_score'].predict(X_test)
predictions['total_score'] = models['total_score'].predict(X_test)

# Calculate metrics
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{model_name} Performance:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.3f}")
    
    return mae, rmse, r2

# Evaluate all models
evaluate_model(y_home_test, predictions['home_score'], "Home Score Model")
evaluate_model(y_away_test, predictions['away_score'], "Away Score Model")
evaluate_model(y_total_test, predictions['total_score'], "Total Score Model")

# Create results dataframe
results_df = modeling_data[test_mask].copy()
results_df['pred_home_score'] = predictions['home_score']
results_df['pred_away_score'] = predictions['away_score']
results_df['pred_total_score'] = predictions['total_score']

print(f"\nSample predictions:")
print(results_df[['home_team', 'away_team', 'home_score', 'away_score', 
                  'pred_home_score', 'pred_away_score']].head())

### Visualize Model Performance

In [ ]:
# Create visualization plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Home Score Predictions
axes[0,0].scatter(y_home_test, predictions['home_score'], alpha=0.6)
axes[0,0].plot([y_home_test.min(), y_home_test.max()], [y_home_test.min(), y_home_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual Home Score')
axes[0,0].set_ylabel('Predicted Home Score')
axes[0,0].set_title('Home Score Predictions')

# Away Score Predictions
axes[0,1].scatter(y_away_test, predictions['away_score'], alpha=0.6)
axes[0,1].plot([y_away_test.min(), y_away_test.max()], [y_away_test.min(), y_away_test.max()], 'r--', lw=2)
axes[0,1].set_xlabel('Actual Away Score')
axes[0,1].set_ylabel('Predicted Away Score')
axes[0,1].set_title('Away Score Predictions')

# Total Score Predictions
axes[1,0].scatter(y_total_test, predictions['total_score'], alpha=0.6)
axes[1,0].plot([y_total_test.min(), y_total_test.max()], [y_total_test.min(), y_total_test.max()], 'r--', lw=2)
axes[1,0].set_xlabel('Actual Total Score')
axes[1,0].set_ylabel('Predicted Total Score')
axes[1,0].set_title('Total Score Predictions')

# Feature Importance
importance = models['home_score'].feature_importances_
feature_imp = pd.DataFrame({'feature': feature_columns, 'importance': importance})
feature_imp = feature_imp.sort_values('importance', ascending=True)

axes[1,1].barh(range(len(feature_imp)), feature_imp['importance'])
axes[1,1].set_yticks(range(len(feature_imp)))
axes[1,1].set_yticklabels(feature_imp['feature'])
axes[1,1].set_xlabel('Feature Importance')
axes[1,1].set_title('Feature Importance (Home Score Model)')

plt.tight_layout()
plt.show()

### Make Predictions for Future Games

In [ ]:
# Function to make predictions for new games
def predict_game(home_team, away_team, week, season, team_perf_data):
    """
    Predict scores for a specific matchup
    """
    # Get recent performance for both teams
    home_recent = team_perf_data[
        (team_perf_data['team'] == home_team) & 
        (team_perf_data['season'] == season) &
        (team_perf_data['week'] <= week)
    ].tail(1)
    
    away_recent = team_perf_data[
        (team_perf_data['team'] == away_team) & 
        (team_perf_data['season'] == season) &
        (team_perf_data['week'] <= week)
    ].tail(1)
    
    if len(home_recent) == 0 or len(away_recent) == 0:
        return None, "Insufficient data for prediction"
    
    # Create feature vector
    features = np.array([[
        week, season,
        home_recent['avg_points_for_L4'].iloc[0] if pd.notna(home_recent['avg_points_for_L4'].iloc[0]) else 21,
        home_recent['avg_points_against_L4'].iloc[0] if pd.notna(home_recent['avg_points_against_L4'].iloc[0]) else 21,
        home_recent['win_rate_L4'].iloc[0] if pd.notna(home_recent['win_rate_L4'].iloc[0]) else 0.5,
        away_recent['avg_points_for_L4'].iloc[0] if pd.notna(away_recent['avg_points_for_L4'].iloc[0]) else 21,
        away_recent['avg_points_against_L4'].iloc[0] if pd.notna(away_recent['avg_points_against_L4'].iloc[0]) else 21,
        away_recent['win_rate_L4'].iloc[0] if pd.notna(away_recent['win_rate_L4'].iloc[0]) else 0.5,
        home_recent['season_points_for_avg'].iloc[0] if pd.notna(home_recent['season_points_for_avg'].iloc[0]) else 21,
        home_recent['season_points_against_avg'].iloc[0] if pd.notna(home_recent['season_points_against_avg'].iloc[0]) else 21,
        away_recent['season_points_for_avg'].iloc[0] if pd.notna(away_recent['season_points_for_avg'].iloc[0]) else 21,
        away_recent['season_points_against_avg'].iloc[0] if pd.notna(away_recent['season_points_against_avg'].iloc[0]) else 21,
        0, 0, 0, 0  # Will calculate these derived features
    ]])
    
    # Calculate derived features
    features[0, 12] = features[0, 2] - features[0, 3]  # home_strength
    features[0, 13] = features[0, 5] - features[0, 6]  # away_strength
    features[0, 14] = features[0, 12] - features[0, 13]  # strength_diff
    features[0, 15] = features[0, 4] - features[0, 7]  # win_rate_diff
    
    # Make predictions
    pred_home = models['home_score'].predict(features)[0]
    pred_away = models['away_score'].predict(features)[0]
    pred_total = models['total_score'].predict(features)[0]
    
    return {
        'home_team': home_team,
        'away_team': away_team,
        'predicted_home_score': round(pred_home, 1),
        'predicted_away_score': round(pred_away, 1),
        'predicted_total': round(pred_total, 1),
        'predicted_winner': home_team if pred_home > pred_away else away_team,
        'confidence': abs(pred_home - pred_away)
    }, None

# Prediction
print("NFL Game Prediction:")
current_season = 2025
current_week = 2

prediction, error = predict_game('HOU', 'TB', current_week, current_season, team_performance)
if prediction:
    print(f"\n{prediction['home_team']} vs {prediction['away_team']}")
    print(f"Predicted Score: {prediction['home_team']} {prediction['predicted_home_score']} - {prediction['predicted_away_score']} {prediction['away_team']}")
    print(f"Predicted Total: {prediction['predicted_total']}")
    print(f"Predicted Winner: {prediction['predicted_winner']}")
    print(f"Margin: {prediction['confidence']:.1f} points")
else:
    print(f"Error: {error}")

### Batch Predictions for Multiple Games

In [ ]:
# Function to predict multiple games
def predict_multiple_games(games_list, team_perf_data):
    """
    Predict scores for multiple games
    games_list: list of tuples (home_team, away_team, week, season)
    """
    predictions = []
    
    for home_team, away_team, week, season in games_list:
        pred, error = predict_game(home_team, away_team, week, season, team_perf_data)
        if pred:
            predictions.append(pred)
        else:
            print(f"Could not predict {home_team} vs {away_team}: {error}")
    
    return pd.DataFrame(predictions)

# Based on current 2025 NFL season schedule - Week 3 upcoming games (Sept 19-22, 2025)
week_3_games = [
    ('BUF', 'MIA', 3, 2025),  # Thursday Night Football - Sept 19
    ('GB', 'CLE', 3, 2025),   # Sunday games
    ('IND', 'TEN', 3, 2025),
    ('CIN', 'MIN', 3, 2025),
    ('NE', 'PIT', 3, 2025),
    ('TB', 'NYJ', 3, 2025),
    ('WAS', 'LAC', 3, 2025),
    ('PHI', 'KC', 3, 2025),   # Big matchup - Super Bowl rematch
    ('SEA', 'CAR', 3, 2025),
    ('DEN', 'HOU', 3, 2025),
    ('LV', 'SF', 3, 2025),
    ('DAL', 'ARI', 3, 2025),
    ('ATL', 'NO', 3, 2025),
    ('DET', 'CHI', 3, 2025),
    ('LAR', 'NYG', 3, 2025),
    ('BAL', 'JAX', 3, 2025)   # Monday Night Football
]

# Week 4 upcoming games (Sept 26-29, 2025)
week_4_games = [
    ('MIN', 'PIT', 4, 2025),  # International game in Dublin, Ireland
    ('KC', 'BAL', 4, 2025),   # Marquee AFC matchup
    ('BUF', 'CIN', 4, 2025),
    ('GB', 'DEN', 4, 2025),
    ('MIA', 'NE', 4, 2025),
    ('NYJ', 'WAS', 4, 2025),
    ('CLE', 'TB', 4, 2025),
    ('IND', 'LAR', 4, 2025),
    ('CAR', 'ATL', 4, 2025),
    ('TEN', 'HOU', 4, 2025),
    ('SF', 'DAL', 4, 2025),
    ('NYG', 'PHI', 4, 2025),
    ('SEA', 'ARI', 4, 2025),
    ('NO', 'DET', 4, 2025),
    ('JAX', 'LV', 4, 2025),
    ('LAC', 'CHI', 4, 2025)
]

# Week 5 upcoming games (Oct 3-6, 2025)
week_5_games = [
    ('MIN', 'CLE', 5, 2025),  # International game in London
    ('NYJ', 'BUF', 5, 2025),
    ('MIA', 'TB', 5, 2025),
    ('NE', 'IND', 5, 2025),
    ('PIT', 'CIN', 5, 2025),
    ('WAS', 'GB', 5, 2025),
    ('TEN', 'CAR', 5, 2025),
    ('BAL', 'HOU', 5, 2025),
    ('ATL', 'JAX', 5, 2025),
    ('SF', 'NO', 5, 2025),
    ('LAR', 'ARI', 5, 2025),
    ('DET', 'SEA', 5, 2025),
    ('CHI', 'LV', 5, 2025),
    ('DAL', 'NYG', 5, 2025),
    ('PHI', 'LAC', 5, 2025),
    ('DEN', 'KC', 5, 2025)    # AFC West rivalry
]

print("=== PREDICTING WEEK 3 GAMES (Sept 19-22, 2025) ===")
week_3_predictions = predict_multiple_games(week_3_games, team_performance)

if len(week_3_predictions) > 0:
    print("\nWeek 3 Predictions:")
    print(week_3_predictions[['home_team', 'away_team', 'predicted_home_score', 
                             'predicted_away_score', 'predicted_winner', 'confidence']].to_string(index=False))
    
    # Highlight marquee matchups
    print("\n🏈 MARQUEE MATCHUPS:")
    marquee_week3 = week_3_predictions[
        ((week_3_predictions['home_team'] == 'PHI') & (week_3_predictions['away_team'] == 'KC')) |
        ((week_3_predictions['home_team'] == 'BUF') & (week_3_predictions['away_team'] == 'MIA'))
    ]
    if len(marquee_week3) > 0:
        for _, game in marquee_week3.iterrows():
            print(f"{game['home_team']} vs {game['away_team']}: {game['predicted_home_score']}-{game['predicted_away_score']} (Winner: {game['predicted_winner']})")

print("\n" + "="*60)
print("=== PREDICTING WEEK 4 GAMES (Sept 26-29, 2025) ===")
week_4_predictions = predict_multiple_games(week_4_games, team_performance)

if len(week_4_predictions) > 0:
    print("\nWeek 4 Predictions:")
    print(week_4_predictions[['home_team', 'away_team', 'predicted_home_score', 
                             'predicted_away_score', 'predicted_winner', 'confidence']].to_string(index=False))

print("\n" + "="*60)
print("=== PREDICTING WEEK 5 GAMES (Oct 3-6, 2025) ===")
week_5_predictions = predict_multiple_games(week_5_games, team_performance)

if len(week_5_predictions) > 0:
    print("\nWeek 5 Predictions:")
    print(week_5_predictions[['home_team', 'away_team', 'predicted_home_score', 
                             'predicted_away_score', 'predicted_winner', 'confidence']].to_string(index=False))

# Summary statistics
all_predictions = pd.concat([week_3_predictions, week_4_predictions, week_5_predictions], ignore_index=True)

if len(all_predictions) > 0:
    print(f"\n📊 PREDICTION SUMMARY (Weeks 3-5):")
    print(f"Total games predicted: {len(all_predictions)}")
    print(f"Average predicted home score: {all_predictions['predicted_home_score'].mean():.1f}")
    print(f"Average predicted away score: {all_predictions['predicted_away_score'].mean():.1f}")
    print(f"Average predicted total: {all_predictions['predicted_total'].mean():.1f}")
    print(f"Home team predicted to win: {(all_predictions['predicted_home_score'] > all_predictions['predicted_away_score']).sum()} games")
    print(f"Away team predicted to win: {(all_predictions['predicted_away_score'] > all_predictions['predicted_home_score']).sum()} games")
    print(f"Average margin of victory: {all_predictions['confidence'].mean():.1f} points")

### International Games

In [ ]:
# Special focus on International Games in 2025
international_games_2025 = [
    ('MIN', 'PIT', 4, 2025, 'Dublin, Ireland'),      # First NFL game in Ireland
    ('MIN', 'CLE', 5, 2025, 'London, England'),      # London game
    ('DEN', 'NYJ', 6, 2025, 'London, England'),      # Second London game
    ('LAR', 'JAX', 7, 2025, 'London, England'),      # Third London game
    ('ATL', 'IND', 10, 2025, 'Berlin, Germany'),     # First NFL game in Berlin
    ('WAS', 'MIA', 11, 2025, 'Madrid, Spain')        # First NFL game in Spain
]

print("🌍 2025 NFL INTERNATIONAL GAMES PREDICTIONS 🌍")
print("=" * 60)

for home_team, away_team, week, season, location in international_games_2025:
    pred, error = predict_game(home_team, away_team, week, season, team_performance)
    if pred:
        print(f"\n📍 {location} - Week {week}")
        print(f"{pred['home_team']} vs {pred['away_team']}")
        print(f"Predicted Score: {pred['predicted_home_score']:.1f} - {pred['predicted_away_score']:.1f}")
        print(f"Predicted Winner: {pred['predicted_winner']}")
        print(f"Margin: {pred['confidence']:.1f} points")
    else:
        print(f"\n❌ Could not predict {home_team} vs {away_team} in {location}: {error}")

print("\nNote: These international games represent historic firsts for the NFL in Dublin, Berlin, and Madrid!")

### Model Summary and Next Steps

In [ ]:
# Print model summary
print("=== NFL SCORE PREDICTION MODEL SUMMARY ===")
print(f"Training Data: {len(X_train)} games from 1999-{recent_cutoff-1}")
print(f"Test Data: {len(X_test)} games from {recent_cutoff} onwards")
print(f"Features Used: {len(feature_columns)}")

print("\n=== KEY FEATURES ===")
for i, feature in enumerate(feature_columns):
    importance = models['home_score'].feature_importances_[i]
    print(f"{feature}: {importance:.3f}")

print("\n=== MODEL PERFORMANCE ===")
print("Home Score MAE: {:.2f} points".format(mean_absolute_error(y_home_test, predictions['home_score'])))
print("Away Score MAE: {:.2f} points".format(mean_absolute_error(y_away_test, predictions['away_score'])))
print("Total Score MAE: {:.2f} points".format(mean_absolute_error(y_total_test, predictions['total_score'])))

print("\n=== NEXT STEPS FOR IMPROVEMENT ===")
print("1. Add more features: weather, injuries, rest days")
print("2. Include playoff/rivalry game indicators")
print("3. Implement ensemble methods")
print("4. Add player-level statistics")
print("5. Include betting market data")
print("6. Implement time-series cross-validation")

print("\n=== USAGE ===")
print("Use predict_game() function to make predictions for specific matchups")
print("Use predict_multiple_games() for batch predictions")
print("Model is ready for real-time game prediction!")